In [1]:
# Create a clone of the repository in your Google Drive with the and link the folder below.

# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "cv2020project/python/pytorch-cnn-visualizations/src"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

ModuleNotFoundError: No module named 'google.colab'

In [3]:
%cd src
from cnn_layer_visualization import CNNLayerVisualization

/home/aschroeter/cv2020project/python/pytorch-cnn-visualizations/src


In [6]:
import os
import numpy as np

import torch
from torch.optim import Adam
from torchvision import models

from misc_functions import preprocess_image, recreate_image, save_image

cnn_layer = 17
filter_pos = 5
# Fully connected layer is not needed
pretrained_model = models.vgg16(pretrained=True).features
layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos)

# Layer visualization with pytorch hooks
layer_vis.visualise_layer_with_hooks()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/aschroeter/.cache/torch/hub/checkpoints/vgg16-397923af.pth



Iteration: 1 Loss: 1.16
Iteration: 2 Loss: -2.16
Iteration: 3 Loss: -8.99
Iteration: 4 Loss: -16.27
Iteration: 5 Loss: -23.25
Iteration: 6 Loss: -29.73
Iteration: 7 Loss: -35.95
Iteration: 8 Loss: -41.89
Iteration: 9 Loss: -47.57
Iteration: 10 Loss: -53.08
Iteration: 11 Loss: -58.51
Iteration: 12 Loss: -63.84
Iteration: 13 Loss: -69.07
Iteration: 14 Loss: -74.23
Iteration: 15 Loss: -79.33
Iteration: 16 Loss: -84.40
Iteration: 17 Loss: -89.45
Iteration: 18 Loss: -94.45
Iteration: 19 Loss: -99.47
Iteration: 20 Loss: -104.46
Iteration: 21 Loss: -109.45
Iteration: 22 Loss: -114.44
Iteration: 23 Loss: -119.45
Iteration: 24 Loss: -124.48
Iteration: 25 Loss: -129.51
Iteration: 26 Loss: -134.57
Iteration: 27 Loss: -139.64
Iteration: 28 Loss: -144.72
Iteration: 29 Loss: -149.84
Iteration: 30 Loss: -154.99


<h1>Guided Grad-Cam</h1>

In [39]:
from guided_gradcam import guided_grad_cam
import numpy as np

from misc_functions import (get_example_params,
                            convert_to_grayscale,
                            save_gradient_images)
from gradcam import GradCam
from guided_backprop import GuidedBackprop

In [40]:
# Get params
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)

# Grad cam
gcv2 = GradCam(pretrained_model, target_layer=11)
# Generate cam mask
cam = gcv2.generate_cam(prep_img, target_class)
print('Grad cam completed')

# Guided backprop
GBP = GuidedBackprop(pretrained_model)
# Get gradients
guided_grads = GBP.generate_gradients(prep_img, target_class)
print('Guided backpropagation completed')

# Guided Grad cam
cam_gb = guided_grad_cam(cam, guided_grads)
save_gradient_images(cam_gb, file_name_to_export + '_GGrad_Cam')
grayscale_cam_gb = convert_to_grayscale(cam_gb)
save_gradient_images(grayscale_cam_gb, file_name_to_export + '_GGrad_Cam_gray')
print('Guided grad cam completed')

Grad cam completed
Guided backpropagation completed
Guided grad cam completed


<h1>Guided Backprop</h1>

In [12]:
from guided_backprop import GuidedBackprop

import torch
from torch.nn import ReLU

from misc_functions import (get_example_params,
                            convert_to_grayscale,
                            save_gradient_images,
                            get_positive_negative_saliency)

In [13]:
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)

# Guided backprop
GBP = GuidedBackprop(pretrained_model)
# Get gradients
guided_grads = GBP.generate_gradients(prep_img, target_class)
# Save colored gradients
save_gradient_images(guided_grads, file_name_to_export + '_Guided_BP_color')
# Convert to grayscale
grayscale_guided_grads = convert_to_grayscale(guided_grads)
# Save grayscale gradients
save_gradient_images(grayscale_guided_grads, file_name_to_export + '_Guided_BP_gray')
# Positive and negative saliency maps
pos_sal, neg_sal = get_positive_negative_saliency(guided_grads)
save_gradient_images(pos_sal, file_name_to_export + '_pos_sal')
save_gradient_images(neg_sal, file_name_to_export + '_neg_sal')
print('Guided backprop completed')


Guided backprop completed


<h1>Score Cam - Score-weighted Class Activation Heatmap on Image (Score-CAM)</h1>

In [57]:
from scorecam import *
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F

from misc_functions import get_example_params, save_class_activation_images

In [58]:
# Get params
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)

In [59]:
# Score cam
score_cam = ScoreCam(pretrained_model, target_layer=11)

In [60]:
# Generate cam mask
cam = score_cam.generate_cam(prep_img, target_class)

In [61]:
# Save mask
save_class_activation_images(original_image, cam, file_name_to_export)
print('Score cam completed')

Score cam completed


![Snake](../results/snake_Cam_On_Image.png)
![Snake](../results/snake_Cam_Heatmap.png)
![Snake](../results/snake_Cam_Grayscale.png)

<h1>Grad Cam - Gradient-weighted Class Activation Heatmap on Image (Grad-CAM)</h1>

In [63]:
from gradcam import *
from PIL import Image
import numpy as np
import torch

from misc_functions import get_example_params, save_class_activation_images

In [66]:
# Get params
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)
# Grad cam
grad_cam = GradCam(pretrained_model, target_layer=11)
# Generate cam mask
cam = grad_cam.generate_cam(prep_img, target_class)
# Save mask
save_class_activation_images(original_image, cam, file_name_to_export)
print('Grad cam completed')

Grad cam completed


![Snake](../results/snake_Cam_On_Image.png)
![Snake](../results/snake_Cam_Heatmap.png)
![Snake](../results/snake_Cam_Grayscale.png)


In [78]:
from gradcam import *
from PIL import Image
import numpy as np
import torch

from misc_functions import get_example_params, save_class_activation_images

In [79]:
# Get params
target_example = 0  # Snake
(original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
    get_example_params(target_example)
# Grad cam
grad_cam = GradCam(pretrained_model, target_layer=11)
# Generate cam mask
cam = grad_cam.generate_cam(prep_img, target_class)
# Save mask
save_class_activation_images(original_image, cam, file_name_to_export)
print('Grad cam completed')

Grad cam completed


![Snake](../results/snake_Cam_On_Image.png)
![Snake](../results/snake_Cam_Heatmap.png)
![Snake](../results/snake_Cam_Grayscale.png)
